In [ ]:
# 1. Imports
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from scipy.optimize import minimize
from sklearn.preprocessing import label_binarize

# 2. Load data

df = pd.read_csv("/content/training-data - Copy - Copy.csv")


# 3. Feature engineering

df['Lesion-volume-log'] = np.log1p(df['Lesion-volume'])
df['NIHSS_x_Age'] = df['NIHSS'] * df['Age']
df['LesionVol_x_Side'] = df['Lesion-volume-log'] * df['Lesion side']
df['Glucose_x_Lesion'] = df['Glucose'] * df['Lesion-volume-log']

manual_features = [
    'Lesion-volume-log', 'NIHSS', 'Age', 'Lesion side', 'Glucose',
    'Previous Stroke', 'BMI', 'Triglycerides', 'Obesity',
    'Creatinine', 'Sex', 'Systolic',
    'NIHSS_x_Age', 'LesionVol_x_Side', 'Glucose_x_Lesion'
]
categorical_feats = ['Lesion side', 'Previous Stroke', 'Sex']

# Drop samples missing mRS
df = df.dropna(subset=['90days-mRS'])

X = df[manual_features]
y_mRS = df['90days-mRS'].astype(int)
y_type = df['lesion type'].astype(int)

cont_feats = [f for f in manual_features if f not in categorical_feats]


# 4. XGBoost parameters & Monotonic Constraints

# Define monotonic constraints based on feature order
monotonic_constraints = [
    +1, # Lesion-volume-log
    +1, # NIHSS
    +1, # Age
    0,  # Lesion side
    +1, # Glucose
    0,  # Previous Stroke
    0,  # BMI
    0,  # Triglycerides
    0,  # Obesity
    +1, # Creatinine
    0,  # Sex
    +1, # Systolic
    +1, # NIHSS_x_Age
    0,  # LesionVol_x_Side
    +1, # Glucose_x_Lesion
]

params_mrs = {
    'objective': 'binary:logistic',
    'learning_rate': 0.018,
    'max_depth': 4,
    'n_estimators': 5000,
    'eval_metric': 'logloss',
    'tree_method': 'hist',
    'seed': 42,
    # *** ADDED MONOTONIC CONSTRAINTS ***
    'monotone_constraints': tuple(monotonic_constraints)
}

params_type = {
    'objective': 'binary:logistic',
    'learning_rate': 0.02,
    'max_depth': 5,
    'n_estimators': 3000,
    'eval_metric': 'logloss',
    'tree_method': 'hist',
    'seed': 42
}


# 5. Functions (Dynamic Weights & QWK Tuning)

def train_ordinal_xgb(X_train, y_train, X_val, y_val, params):
    """Trains the mRS head with DYNAMIC scale_pos_weight for each ordinal cut."""
    n_classes = int(y_train.max()) + 1
    preds_val = np.zeros((X_val.shape[0], n_classes-1))

    for k in range(n_classes-1):
        y_bin_train = (y_train > k).astype(int)
        y_bin_val = (y_val > k).astype(int)

        current_params = params.copy()

        # DYNAMIC SCALE_POS_WEIGHT (MRS HEAD)
        pos_count = np.sum(y_bin_train)
        neg_count = len(y_bin_train) - pos_count

        if pos_count > 0 and neg_count > 0:
            current_params['scale_pos_weight'] = neg_count / pos_count

        dtrain = xgb.DMatrix(X_train, label=y_bin_train)
        dvalid = xgb.DMatrix(X_val, label=y_bin_val)

        model = xgb.train(
            current_params,
            dtrain,
            num_boost_round=params['n_estimators'],
            evals=[(dvalid, 'valid')],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        preds_val[:, k] = model.predict(dvalid)
    return preds_val

def train_type_xgb(X_train, y_train, X_val, y_val, params):
    """Trains the Lesion Type head with DYNAMIC scale_pos_weight."""
    current_params = params.copy()

    # DYNAMIC SCALE_POS_WEIGHT (TYPE HEAD)
    pos_count = np.sum(y_train)
    neg_count = len(y_train) - pos_count

    if pos_count > 0 and neg_count > 0:
        current_params['scale_pos_weight'] = neg_count / pos_count

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        current_params,
        dtrain,
        num_boost_round=params['n_estimators'],
        evals=[(dvalid, 'valid')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    return model.predict(dvalid)

def apply_thresholds(pred_matrix, thresholds):
    """Counts the number of exceeded probability thresholds to get the mRS prediction."""
    n_samples, n_preds = pred_matrix.shape
    final_preds = np.zeros(n_samples, dtype=int)
    for i in range(n_samples):
        score = 0
        for k in range(n_preds):
            if pred_matrix[i, k] >= thresholds[k]:
                score += 1
        final_preds[i] = score
    return final_preds

def tune_thresholds(y_true, prob_matrix):
    """Uses Powell minimization to find thresholds that maximize QWK (Minimize -QWK)."""
    n_cuts = prob_matrix.shape[1]
    initial_thresholds = np.full(n_cuts, 0.5)

    def loss(thresholds):
        preds = apply_thresholds(prob_matrix, thresholds)
        return -cohen_kappa_score(y_true, preds, weights='quadratic')

    # Use Powell method with bounds [0, 1] for robust threshold optimization
    result = minimize(loss, initial_thresholds, method='Powell', bounds=[(0, 1)]*n_cuts)
    return result.x


# 6. 5-Fold Cross-Validation (Leakage Fixed)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_mrs = np.zeros((X.shape[0], 6))
oof_type = np.zeros(X.shape[0])

imputer = SimpleImputer(strategy='median')
scaler = RobustScaler()
impute_scale_cols = cont_feats

for train_idx, val_idx in kf.split(X):
    # Use .copy() to prevent SettingWithCopyWarning
    Xtr, Xv = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
    ytr_mRS, yv_mRS = y_mRS.iloc[train_idx], y_mRS.iloc[val_idx]
    ytr_type, yv_type = y_type.iloc[train_idx], y_type.iloc[val_idx]

    # --- LEAKAGE FIX ZONE: Imputation and Scaling INSIDE THE CV LOOP ---
    # 1. Imputation (Fit on Train, Transform All)
    Xtr.loc[:, impute_scale_cols] = imputer.fit_transform(Xtr[impute_scale_cols])
    Xv.loc[:, impute_scale_cols] = imputer.transform(Xv[impute_scale_cols])

    # 2. Scaling (Fit on Train, Transform All)
    Xtr.loc[:, impute_scale_cols] = scaler.fit_transform(Xtr[impute_scale_cols])
    Xv.loc[:, impute_scale_cols] = scaler.transform(Xv[impute_scale_cols])
    # --- END LEAKAGE FIX ZONE ---

    # Train ordinal mRS (FIXED CALL: passing yv_mRS)
    oof_mrs[val_idx] = train_ordinal_xgb(Xtr, ytr_mRS, Xv, yv_mRS, params_mrs)

    # Train binary Type
    oof_type[val_idx] = train_type_xgb(Xtr, ytr_type, Xv, yv_type, params_type)


# 7. Threshold Tuning & Metrics

thresholds = tune_thresholds(y_mRS.values, oof_mrs)
final_pred_mrs = apply_thresholds(oof_mrs, thresholds)
# Assuming 0.5 threshold for Type for simplicity (Tuning is preferred)
final_pred_type = (oof_type > 0.5).astype(int)

print("\n" + "="*50)
print("FINAL METRICS (XGBOOST DUAL-HEAD: MONOTONIC + DYNAMIC WEIGHTS)")
print("="*50)
print(f"FINAL OOF QWK (mRS) = {cohen_kappa_score(y_mRS, final_pred_mrs, weights='quadratic'):.4f}")
print(f"FINAL OOF ACC (Type) = {accuracy_score(y_type, final_pred_type):.4f}")
print(f"Binary Type ROC-AUC = {roc_auc_score(y_type, oof_type):.4f}")

try:
    # Calculation for mRS multi-class ROC-AUC
    mrs_auc_approx = roc_auc_score(y_mRS, oof_mrs.mean(axis=1), average='weighted')
    print(f"mRS multi-class ROC-AUC (Approx): {mrs_auc_approx:.4f}")
except Exception as e:
    print(f"mRS AUC could not be computed easily. Error: {e}")

print("\nBest Thresholds (mRS k=0 to 5):", np.round(thresholds, 3))
print("="*50)